In [37]:
import pandas as pd

In [38]:
df_old_metrics: pd.DataFrame = pd.read_csv('../data/embedded_datasets/metrics_dataset.csv')
print(df_old_metrics.shape)
df_old_metrics.head()

(2242, 32)


,sample_id,severity,constructor,line,cbo,wmc,rfc,loc,returnsQty,variablesQty,...,maxNestedBlocksQty,anonymousClassesQty,innerClassesQty,lambdasQty,uniqueWordsQty,modifiers,logStatementsQty,hasJavaDoc,from_project,label
0,4256584,major,0,65,1,1,5,7,0,0,...,0,0,0,0,32,1,0,1,1,1
1,8922371,major,0,733,3,15,14,71,0,1,...,3,0,0,0,5,1,0,0,1,1
2,8653310,major,0,105,10,7,73,24,1,6,...,3,0,0,0,32,1,0,1,1,1
3,4734605,major,0,929,6,4,28,20,0,6,...,1,0,0,1,39,2,2,0,0,1
4,4514232,major,0,100,15,23,19,65,0,20,...,3,0,0,0,58,33,0,0,1,1


In [39]:
df_pmd: pd.DataFrame = pd.read_csv('../data/embedded_datasets/mlcq_pmd.csv')
df_pmd = df_pmd.loc[df_pmd['smell'] == 'feature envy']
df_pmd = df_pmd.loc[df_pmd['language'] == 'java']
df_pmd = df_pmd.dropna(axis='columns')
df_pmd = df_pmd.drop(columns=['id','reviewer_id','col1','smell','review_timestamp','type','name','analysis_timestamp','name..10','language','repository','commit_hash','sample_id..14','severity'])
df_pmd = df_pmd.drop_duplicates()
print(df_pmd.shape)
df_pmd.head()

(2415, 7)


,sample_id,abc_size,arity,block_nesting,cyclo,lines_of_code,return_values
3,3698602,1.000000,0.0,0.0,0.0,3,0.0
4,3698665,2.000000,1.0,0.0,0.0,4,0.0
6,3698860,0.000000,0.0,0.0,0.0,14,0.0
9,3699227,0.000000,0.0,0.0,0.0,3,0.0
11,3699521,4.472136,0.0,0.0,0.0,9,0.0


In [40]:
df = pd.merge(df_old_metrics, df_pmd, on='sample_id')
print(df.shape)
df.head()

(2242, 38)


,sample_id,severity,constructor,line,cbo,wmc,rfc,loc,returnsQty,variablesQty,...,logStatementsQty,hasJavaDoc,from_project,label,abc_size,arity,block_nesting,cyclo,lines_of_code,return_values
0,4256584,major,0,65,1,1,5,7,0,0,...,0,1,1,1,5.000000,4.0,0.0,0.0,9,0.0
1,8922371,major,0,733,3,15,14,71,0,1,...,0,0,1,1,26.248809,2.0,2.0,8.0,66,0.0
2,8653310,major,0,105,10,7,73,24,1,6,...,0,1,1,1,79.611557,3.0,2.0,4.0,86,0.0
3,4734605,major,0,929,6,4,28,20,0,6,...,2,0,0,1,33.867388,5.0,1.0,3.0,27,0.0
4,4514232,major,0,100,15,23,19,65,0,20,...,0,0,1,1,61.554854,1.0,3.0,14.0,105,0.0


In [41]:
pd.to_pickle(df, '../data/embedded_datasets/metrics_pmd_merged.pkl')
del df, df_old_metrics, df_pmd

# Merge embedding

In [42]:
df_metrics = pd.read_pickle('../data/embedded_datasets/metrics_pmd_merged.pkl')

aggregations = ['whole', 'avg', 'sum']
for agg in aggregations:
    df_t5p = pd.read_pickle(f'../data/embedded_datasets/T5_plus_{agg}.pkl')
    df_t5p = df_t5p.drop(columns=['severity', 'label'])
    df = pd.merge(df_t5p, df_metrics, on='sample_id')
    pd.to_pickle(df, f'./embedded_datasets/{agg}_pmd_merged.pkl')

In [43]:
df = pd.read_pickle('./embedded_datasets/whole_merged.pkl')
df.head()

,sample_id,em_1,em_2,em_3,em_4,em_5,em_6,em_7,em_8,em_9,...,maxNestedBlocksQty,anonymousClassesQty,innerClassesQty,lambdasQty,uniqueWordsQty,modifiers,logStatementsQty,hasJavaDoc,from_project,label
0,4256584,-0.082731,0.025804,-0.002167,-0.054238,0.071943,0.074374,-0.005104,-0.019794,-0.081045,...,0,0,0,0,32,1,0,1,1,1
1,8922371,-0.055804,-0.064806,-0.017028,0.074632,-0.051724,0.096277,-0.081308,-0.036029,0.139101,...,3,0,0,0,5,1,0,0,1,1
2,8653310,-0.063023,-0.066769,-0.081059,-0.014370,-0.014759,0.080498,0.039054,-0.039447,-0.001215,...,3,0,0,0,32,1,0,1,1,1
3,4734605,-0.008280,-0.101544,-0.059682,0.047641,-0.065149,-0.000050,-0.062970,0.018910,0.166090,...,1,0,0,1,39,2,2,0,0,1
4,4514232,0.062265,0.030177,-0.096552,-0.037738,-0.126714,0.041150,-0.012203,0.035476,-0.039626,...,3,0,0,0,58,33,0,0,1,1
